In [1]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
R.<x> = PolynomialRing(QQ)
Scplx= PowerSeriesRing(CC,'x',default_prec=300)
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)
def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)
def j(n,m):return L(xjNew(n,m)/x)
def H4(n,m):
    jay=j(n+1,m)
    djay=x*derivative(jay,x) # bc variable is tau, not x; chain rule.
    numerator = djay^2
    denominator = jay*(jay-2^6*m^3)
    return ((S(numerator/denominator)^(1/(m-2))).O(n+1)).polynomial()
def H6(n,m): #different than in draft 4mar21 and Mathematica.
    def base6(n,m):
        jay=j(n+1,m)
        num=(x*derivative(jay,x))^m # chain rule; variable is tau not x.
        den = expand(jay^(m-1)*(j(n+1,m)-2^6*m^3))
        return L((-1)^m*(num/den)).O(n+1)
    return S(base6(n,m)^(1/(m-2))).polynomial()
def polynomialCoefficient(n,poly):
    return poly.list()[n]
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

In [4]:
j(5,3)

x^-1 + 744 + 196884*x + 21493760*x^2 + 864299970*x^3 + 20245856256*x^4

In [17]:
print(H6(5,3));print(type(H6(5,3)))

-1575504*x^5 - 532728*x^4 - 122976*x^3 - 16632*x^2 - 504*x + 1
<type 'sage.rings.polynomial.polynomial_rational_flint.Polynomial_rational_flint'>


In [15]:
H6(5,4)

-10569646080*x^5 - 193986560*x^4 - 9175040*x^3 - 102400*x^2 - 1280*x + 1

In [7]:
H6(5,3)
    

-1575504*x^5 - 532728*x^4 - 122976*x^3 - 16632*x^2 - 504*x + 1

In [6]:
# correct signs, at the expense of deviating from the
# normalization in the 4mar21 draft.
poly=H6(5,3)
plist=poly.list()
for k in [1..5]:
    print [k,-504*sigma(k,5),plist[k]]

[1, -504, -504]
[2, -16632, -16632]
[3, -122976, -122976]
[4, -532728, -532728]
[5, -1575504, -1575504]


In [4]:
diffs=[]
poly=H6(100,3)
plist=poly.list()
for k in [1..poly.degree()]:
    diffs=diffs+[-504*sigma(k,5)-plist[k]]
print diffs

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [11]:
import pickle
wfile = open('/Users/barrybrent/7mar21no2.txt','a')
data=[]
for m in [3..9]:
    if 1 in divisors(m):
        print(m)
    poly=H6(12,m)
    data=data+[[m,poly.list()]]
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

3
4
5
6
7
8
9


In [12]:
import pickle
rfile = open('/Users/barrybrent/7mar21no2.txt','r')
data=[]
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
print(s)

[[3, [1, -504, -16632, -122976, -532728, -1575504, -4058208, -8471232, -17047800, -29883672, -51991632, -81170208, -129985632]], [4, [1, -1280, -102400, -9175040, -193986560, -10569646080, -187904819200, -7387343749120, -100674033418240, -4161651511132160, -55415386039910400, -1874623344892968960, -23328646069779169280]], [5, [1, -2600, -351000, -276380000/3, -29091725000/3, -461573190000, -884466669580000/9, -42731073946600000/9, -2925509932307625000/7, -22528177776031841875000/567, -764196116461702063750000/567, -4519028573198488719500000/21, -20593777488239481724217500000/1701]], [6, [1, -4608, -884736, -528482304, -125627793408, -14843406974976, -3087428650795008, -1013309916158361600, -66148871326817845248, -20254524992933087674368, -3060093480899530098475008, -326409971295949877170667520, -63367055760920402821398921216]], [7, [1, -7448, -1824760, -2226207200, -932746146360, -1088046362821712/5, -281820099671626208/5, -36817178522254305728, -9662291816318397239480, -12139508201766

In [3]:
# Had to shut down & restart to avoid an error message
# bc two looping routines involving H6 throws one..
import pickle
wfile = open('/Users/barrybrent/7mar21no3.txt','a')
data=[]
for m in [3..9]:
    if 1 in divisors(m):
        print(m)
    poly=H6(12,m)
    data=data+[[m,poly]]
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

3
4
5
6
7
8
9


In [4]:
import pickle
rfile = open('/Users/barrybrent/7mar21no3.txt','r')
data=[]
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
print(s)

[[3, -129985632*x^12 - 81170208*x^11 - 51991632*x^10 - 29883672*x^9 - 17047800*x^8 - 8471232*x^7 - 4058208*x^6 - 1575504*x^5 - 532728*x^4 - 122976*x^3 - 16632*x^2 - 504*x + 1], [4, -23328646069779169280*x^12 - 1874623344892968960*x^11 - 55415386039910400*x^10 - 4161651511132160*x^9 - 100674033418240*x^8 - 7387343749120*x^7 - 187904819200*x^6 - 10569646080*x^5 - 193986560*x^4 - 9175040*x^3 - 102400*x^2 - 1280*x + 1], [5, -20593777488239481724217500000/1701*x^12 - 4519028573198488719500000/21*x^11 - 764196116461702063750000/567*x^10 - 22528177776031841875000/567*x^9 - 2925509932307625000/7*x^8 - 42731073946600000/9*x^7 - 884466669580000/9*x^6 - 461573190000*x^5 - 29091725000/3*x^4 - 276380000/3*x^3 - 351000*x^2 - 2600*x + 1], [6, -63367055760920402821398921216*x^12 - 326409971295949877170667520*x^11 - 3060093480899530098475008*x^10 - 20254524992933087674368*x^9 - 66148871326817845248*x^8 - 1013309916158361600*x^7 - 3087428650795008*x^6 - 14843406974976*x^5 - 125627793408*x^4 - 528482304*

In [18]:
data=[]
for m in [3..10]:
    print m
    data=data+[[m,H6(5,m)]]
import pickle
wfile = open('/Users/barrybrent/8mar21no11.txt','a')
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

3
4
5
6
7
8
9
10


In [20]:
import pickle
rfile = open('/Users/barrybrent/8mar21no11.txt','r')
data=[]
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for k in  [0..3]:
    print s[k][1]
    print "--------------------------------------------------------------------------------------"

-1575504*x^5 - 532728*x^4 - 122976*x^3 - 16632*x^2 - 504*x + 1
--------------------------------------------------------------------------------------
-10569646080*x^5 - 193986560*x^4 - 9175040*x^3 - 102400*x^2 - 1280*x + 1
--------------------------------------------------------------------------------------
-461573190000*x^5 - 29091725000/3*x^4 - 276380000/3*x^3 - 351000*x^2 - 2600*x + 1
--------------------------------------------------------------------------------------
-14843406974976*x^5 - 125627793408*x^4 - 528482304*x^3 - 884736*x^2 - 4608*x + 1
--------------------------------------------------------------------------------------


In [2]:
data=[]
for m in [3..400]:
    if 10 in divisors(m):
        print m
    data=data+[[m,H6(100,m)]]
import pickle
wfile = open('/Users/barrybrent/8mar21no13.txt','a')
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

10
20
30
40
50
60
70
80
90
100
110
120


KeyboardInterrupt: 

In [3]:
len(data)

118

In [5]:
import pickle
wfile = open('/Users/barrybrent/8mar21no13.txt','a')
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

In [4]:
import pickle
rfile = open('/Users/barrybrent/8mar21no13.txt','r')
data=[]
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
lnth=len(s)
s[lnth-1][0]

120

In [2]:
import pickle
wfile = open('/Users/barrybrent/9mar21no3.txt','w')
data=[]
for m in [121..400]:
    print(m)
    data=data+[[m,H6(100,m)]]
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
